In [1]:
from esda.moran import Moran
import geopandas as gpd
import numpy as np
import pandas as pd
from libpysal.weights import DistanceBand, KNN
from matplotlib import colors
import matplotlib.pyplot as plt
from splot.esda import plot_moran
import spreg

from statshelper import q_q_plot

In [2]:
df = pd.read_csv("./data/processed/Cleaned_Chicago_Sales.csv")
df.head()

,Unnamed: 0,PIN,Property Class,Neighborhood Code,Land Square Feet,Town Code,Type of Residence,Apartments,Wall Material,Roof Material,...,Bool-Type of Residence-1,Bool-Type of Residence-2,Bool-Type of Residence-4,Bool-Type of Residence-8,Bool-Type of Residence-7,Bool-Type of Residence-6,Bool-Wall Material-2,Bool-Wall Material-3,Bool-Wall Material-1,Bool-Wall Material-4
0,2,16094150130000,211,13,-0.297580,77,3.0,6.0,2.0,2.0,...,0,0,0,0,0,0,1,0,0,0
1,5,4252000820000,204,100,2.868180,25,5.0,0.0,2.0,4.0,...,0,0,0,0,0,0,1,0,0,0
2,8,14322110150000,208,12,-0.596010,74,3.0,0.0,2.0,6.0,...,0,0,0,0,0,0,1,0,0,0
3,9,27021200080000,204,34,1.698873,28,1.0,0.0,3.0,1.0,...,1,0,0,0,0,0,0,1,0,0
4,11,13121080620000,204,42,0.515768,71,1.0,0.0,2.0,1.0,...,1,0,0,0,0,0,1,0,0,0


In [3]:
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.Longitude, df.Latitude))

In [7]:
i_vars = [
    'Age',
    'Bedrooms',  
    'Building Square Feet', 
    'Estimate (Building)', 
    'Estimate (Land)', 
    'Fireplaces', 
    'Full Baths',
    'Garage 1 Area',
    'Garage 2 Area', 
    'Half Baths', 
    'Land Square Feet',
    'Rooms',
    'Sale Quarter'
    ]
dep_var = ['Sale Price']
other_to_keep = ["PIN", "Neighborhood Code", "Town Code", "geometry"]
my_rows = i_vars+dep_var+other_to_keep
X = gdf[i_vars].values
y = gdf[dep_var].values.ravel()
gdf = gdf[my_rows]

In [8]:
gdf.describe()

,Age,Bedrooms,Building Square Feet,Estimate (Building),Estimate (Land),Fireplaces,Full Baths,Garage 1 Area,Garage 2 Area,Half Baths,Land Square Feet,Rooms,Sale Quarter,Sale Price,PIN,Neighborhood Code,Town Code
count,3.242610e+05,3.242610e+05,3.242610e+05,3.242610e+05,3.242610e+05,3.242610e+05,3.242610e+05,3.242610e+05,3.242610e+05,3.242610e+05,3.242610e+05,3.242610e+05,324261.000000,3.242610e+05,3.242610e+05,324261.000000,324261.000000
mean,-7.120576e-10,-1.092022e-08,7.167054e-10,1.437860e-10,5.046344e-10,1.617216e-09,-3.436738e-09,1.683520e-09,1.066499e-09,-2.931867e-08,-3.191524e-10,-1.353879e-08,78.852471,-5.396458e-10,1.651628e+13,108.992253,44.675265
std,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,9.999999e-01,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,7.987350,1.000000e+00,8.069302e+12,100.150467,23.754169
min,-2.077472e+00,-3.513837e+00,-3.056954e+00,-1.016233e+01,-9.972469e+00,-5.779852e-01,-8.912787e-01,-2.100215e+00,-4.176153e-02,-8.080311e-01,-5.640353e+00,-3.444030e+00,65.000000,-3.193263e+00,1.011000e+12,10.000000,10.000000
25%,-6.126325e-01,-2.926810e-01,-7.569215e-01,-6.368033e-01,-6.296315e-01,-5.779852e-01,-8.912787e-01,-7.751074e-01,-4.176153e-02,-8.080311e-01,-5.930628e-01,-7.957876e-01,72.000000,-4.957421e-01,1.036208e+13,34.000000,24.000000
50%,-1.668120e-01,-2.926810e-01,-1.910355e-01,-1.657273e-02,-9.333429e-02,-5.779852e-01,-8.912787e-01,5.500000e-01,-4.176153e-02,-8.080311e-01,-9.124129e-02,-2.688460e-01,79.000000,1.045530e-01,1.608316e+13,80.000000,37.000000
75%,7.885178e-01,5.508094e-01,6.588384e-01,5.936153e-01,5.069814e-01,1.270738e+00,6.356368e-01,5.500000e-01,-4.176153e-02,9.666556e-01,6.044575e-01,5.626062e-01,86.000000,6.221981e-01,2.311401e+13,150.000000,71.000000
max,3.654507e+00,9.679707e+00,5.458316e+00,5.511245e+00,7.154818e+00,3.119462e+00,7.342334e+00,5.500000e-01,2.598733e+01,1.516415e+01,9.886416e+00,1.040469e+01,92.000000,4.165429e+00,3.332302e+13,600.000000,77.000000


In [9]:
# determine weight matrix
w = KNN.from_dataframe(gdf, k=128) # needed k large enough that it was connected
#w = DistanceBand.from_dataframe(gdf,threshold=.0075) # needed k large enough that it was connected

In [10]:
w.transform = 'R'

In [ ]:
sem = spreg.ML_Error(y, X, w=w, name_x=i_vars, name_y="Sale Price", method="LU")

/home/dijkstra/.local/lib/python3.8/site-packages/scipy/optimize/_minimize.py:777: RuntimeWarning: Method 'bounded' does not support relative tolerance in x; defaulting to absolute tolerance.
  warn("Method 'bounded' does not support relative tolerance in x; "


In [ ]:
print(sem.summary)

In [ ]:
sem_moran = Moran(sem.e_filtered, w, permutations=199)
plot_moran(sem_moran, zstandard=True, figsize=(10,4))

In [ ]:
data = sem.e_filtered.reshape(-1)
_min, _max = min(data), max(data)
print(_min, _max)
_width = (_max-_min)
norm = colors.BoundaryNorm(np.arange(_min, int(_max), _width/7),ncolors=256)
ax = gdf.plot(column=sem.e_filtered.reshape(-1),legend=True,figsize=(15,8), norm=norm, linewidth=0.0)
plt.title("Map of residuals of the SEM model",fontsize=15)
plt.show()